### **Code : (3) 자금 관리 패턴 모델링**

- Project : 2024 데이터바우처 지원사업
- Writer : Donghyeon Kim
- Update : 2024.10.05.

#### **0. 라이브러리 및 초기 경로 설정**

In [1]:
import os
import pandas as pd
import numpy as np
import xlrd
import warnings
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from openpyxl import Workbook, load_workbook
from openpyxl.styles import PatternFill
from matplotlib import font_manager, rc, rcParams, ticker
from datetime import datetime, date
from tqdm import tqdm

In [2]:
dr = 'C:/'
folder_1 = 'Users/USER/Dropbox/8. 회사업무/1. 산학협력프로젝트/2024년/9. [선정] 2024 데이터바우처 지원사업/7. 분석'
root = dr + folder_1

folder_2 = '1_rawdata'
folder_path = root + '/' + folder_2

os.chdir(folder_path)

In [3]:
# 회사이름
company_name = os.listdir()[6] # Available Index : 0 ~ 9 (Total : 10)

# 회사이름 포함 경로
final_path = folder_path + '/' + company_name
os.chdir(final_path)

---

#### **1. 입금내역 Data Load**

In [4]:
# '입금내역_수납확인' 파일명
file_name = [file for file in os.listdir() if '입금내역_수납확인' in file] # 입금내역 파일 1개

# Data Frame
df = pd.read_excel(file_name[0], skiprows=1)

# Data Frame Head(2023)
df_without_account = df.drop(columns=['계좌번호', '계좌적요', '거래처'])
df_without_account[df_without_account['입금일자'].str.contains('2023-12')].head()

,입금일자,은행,입금액,용도,내용,프로젝트/현장,비고,메모
105,2023-12-29,우리은행,660000,매출대금입금,NaN,NaN,2023-11-23 세계2023-12-28 세계,NaN
106,2023-12-28,우리은행,330000,매출대금입금,NaN,NaN,2023-12-20 세계,NaN
107,2023-12-28,우리은행,2607000,매출대금입금,NaN,NaN,2023-11-30 세계,NaN
108,2023-12-28,우리은행,2200000,매출대금입금,NaN,NaN,2023-12-27 세계,NaN
109,2023-12-19,우리은행,440000,매출대금입금,NaN,NaN,2023-12-19 세계,NaN


In [5]:
# 회사명에 따른 결과물 저장 경로
result_root = os.path.join(root, 'Model(3rd)')
company_result_root = os.path.join(result_root, company_name)
if not os.path.isdir(company_result_root):
    os.makedirs(company_result_root)

#### **용도별 현금 유입 주기 및 변동액**

In [6]:
print('용도별 현금 유입 주기 및 변동액 계산 중')

# 1) '입금일자' Datetime 형태로 변경
df['입금일자'] = pd.to_datetime(df['입금일자'].copy())
df_filtered = df[df['입금일자'].dt.year <= 2023].copy()

# 2) '용도', '입금일자' 정렬
sorted_df = df_filtered.sort_values(by=['용도', '입금일자']).copy()

# 3) 용도별 현금 유입 주기 계산
sorted_df['유입주기'] = sorted_df.groupby(['용도'])['입금일자'].diff().dt.days

# 4) 용도별 현금 유입액 및 주기 분석
fluctuation_analysis = sorted_df.groupby('용도').agg({
    '입금액': ['mean', 'std', 'count'],
    '유입주기': 'mean'
}).copy()

# 5) 표준편차 기준 위험도 분류 (표준편차가 클수록 위험도 증가하는 방식)
fluctuation_analysis.columns = ['평균입금액', '입금액표준편차(변동액)', '입금횟수', '평균유입주기(일)']

# 6) 평균유입주기 기준 위험도 평가
# 방법 : 평균유입주기가 긴 경우 위험도를 높게 설정 → 주기가 일정한데도 주기가 길면 위험하다고 판단하는 방식
bins = [0, fluctuation_analysis['평균유입주기(일)'].quantile(0.33), fluctuation_analysis['평균유입주기(일)'].quantile(0.66), float('inf')]
labels = ['낮음', '보통', '높음']
fluctuation_analysis['주기위험도'] = pd.cut(fluctuation_analysis['평균유입주기(일)'], bins=bins, labels=labels)

# 7) 입금횟수 2번 이상만 필터링
fluctuation_analysis_filtered = fluctuation_analysis[fluctuation_analysis['입금횟수'] >= 2].copy()

# 8) '평균입금액', '입금액표준편차', '평균유입주기' 소수점 2자리까지 출력
fluctuation_analysis_filtered['평균입금액'] = fluctuation_analysis_filtered['평균입금액'].round(2)
fluctuation_analysis_filtered['입금액표준편차(변동액)'] = fluctuation_analysis_filtered['입금액표준편차(변동액)'].round(2)
fluctuation_analysis_filtered['평균유입주기(일)'] = fluctuation_analysis_filtered['평균유입주기(일)'].round(2)

# 결과 출력
fluctuation_analysis_filtered

용도별 현금 유입 주기 및 변동액 계산 중


,평균입금액,입금액표준편차(변동액),입금횟수,평균유입주기(일),주기위험도
용도,,,,,
과입금액,5662250.00,7074249.79,2,779.00,높음
국고보조금수익,4500000.00,2121320.34,2,96.00,높음
기타보조금수익,225000.00,0.00,2,24.00,보통
기타수입,639787.50,241755.07,4,173.33,높음
기타환급금수익,279483.33,336391.42,3,11.00,낮음
내사업계좌간 이체,5579838.69,5251176.06,61,18.62,낮음
매출대금입금,1540114.11,1845509.44,964,1.19,낮음
매출대금입금 외,2592500.00,1992040.41,4,256.00,높음
이자수익,36323.27,17010.59,26,43.68,보통


In [13]:
# Model 평가 : 일치율(실제값과 예측치 간 동일한 비율)

# 1) Training Data와 Test Data 나누기 : 80% vs 20%
X = fluctuation_analysis_filtered[['평균입금액', '입금액표준편차(변동액)', '평균유입주기(일)']].copy()  # 입력 변수
y = fluctuation_analysis_filtered['주기위험도'].copy()  # 타겟 변수

# 평균유입주기(일)에 결측값이 있는 경우 위험도를 '높음'으로 처리
X['평균유입주기(일)'] = X['평균유입주기(일)'].fillna(0)  # 결측값을 0으로 대체
y = y.where(X['평균유입주기(일)'] != 0, '높음')  # 평균 유입 주기가 0인 경우 위험도를 '높음'으로 설정

# 2) Training과 Test 데이터 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# 3) 예측 모델 (훈련 데이터의 평균 유입 주기를 기준으로 예측)
# 주기가 짧을수록 위험도가 '낮음', 길수록 위험도가 '높음'이므로 조건을 설정
y_pred = X_test['평균유입주기(일)'].apply(lambda x: '낮음' if x <= X_train['평균유입주기(일)'].mean() else '높음')

# 4) 일치율 평가 (accuracy)
accuracy = accuracy_score(y_test, y_pred)
accuracy_percentage = round(accuracy * 100, 2)  # 정확도를 소수점 2자리 %로 변환
print(f'주기위험도에 대한 일치율: {accuracy_percentage}%')

주기위험도에 대한 일치율: 100.0%


In [14]:
# '용도'를 Columns에 추가하고 기존 인덱스 제거
fluctuation_analysis_filtered = fluctuation_analysis_filtered.reset_index()

# '일치율' 열 추가
fluctuation_analysis_filtered['일치율'] = ''

# 마지막 행에 '일치율 결과'를 추가하고, 계산된 정확도(accuracy) 값을 넣음
fluctuation_analysis_filtered.loc['결과'] = ['', '', '', '', '', '', accuracy_percentage]

# 결과물 최종 경로
output_file = os.path.join(company_result_root, f'{company_name}_입금내역_유입_분석.xlsx')

# 엑셀 파일로 저장
fluctuation_analysis_filtered.to_excel(output_file, index=False)
print('1차 결과 : 유입 주기/변동액 분석 결과가 Excel 파일로 저장되었습니다.')

# 다시 엑셀 파일 불러오기
wb = load_workbook(output_file)
ws = wb.active

# 색상 지정
low_fill = PatternFill(start_color='32CD32', end_color='32CD32', fill_type='solid') # Limegreen
medium_fill = PatternFill(start_color='FFD700', end_color='FFD700', fill_type='solid') # Gold
high_fill = PatternFill(start_color='FF6347', end_color='FF6347', fill_type='solid') # Tomato

# '주기위험도' 열의 색상 적용
for row in range(2, ws.max_row):  # 2번째 행부터 반복(1번째 행 제외)
    risk_level = ws[f'F{row}'].value  # '주기위험도'가 있는 열 = Excel 상에서 F열
    if risk_level == '낮음':
        ws[f'F{row}'].fill = low_fill
    elif risk_level == '보통':
        ws[f'F{row}'].fill = medium_fill
    elif risk_level == '높음':
        ws[f'F{row}'].fill = high_fill

# 수정된 엑셀 파일 저장
wb.save(output_file)

print('2차 결과 : 주기위험도에 색상이 적용되었습니다.')

1차 결과 : 유입 주기/변동액 분석 결과가 Excel 파일로 저장되었습니다.
2차 결과 : 주기위험도에 색상이 적용되었습니다.


---

#### **2. 출금내역 Data Load**

In [16]:
# '출금내역_지급확인' 파일명
file_name = [file for file in os.listdir() if '출금내역_지급확인' in file] # 출금내역 파일 1개

# Data Frame
df = pd.read_excel(file_name[0], skiprows=1)

# Data Frame Head(2023)
df_without_account = df.drop(columns=['계좌번호', '계좌적요', '거래처', '내용'])
df_without_account[df_without_account['출금일자'].str.contains('2023-12')].head()

,출금일자,은행,출금액,용도,프로젝트/현장,비고,메모
181,2023-12-29,우리은행,993800,매입대금지급,NaN,결의서_202312_19\n2023-12-28 세계 수수료 500,NaN
182,2023-12-29,우리은행,880500,매입대금지급,NaN,결의서_202312_18\n2023-12-27 세계 수수료 500,NaN
183,2023-12-29,우리은행,286500,매입대금지급,NaN,결의서_202312_17\n2023-12-26 세계 수수료 500,NaN
184,2023-12-29,우리은행,195560,매입대금지급,NaN,결의서_202312_16\n결의서_202312_16\n2023-12-18 세계202...,NaN
185,2023-12-27,우리은행,200000,일반경조사비,NaN,NaN,NaN


#### **용도별 현금 유출 주기 및 변동액**

In [17]:
print('용도별 현금 유출 주기 및 변동액 계산 중')

# 1) '출금일자' Datetime 형태로 변경
df['출금일자'] = pd.to_datetime(df['출금일자'].copy())
df_filtered = df[df['출금일자'].dt.year <= 2023].copy()

# 2) '용도', '출금일자' 정렬
sorted_df = df_filtered.sort_values(by=['용도', '출금일자']).copy()

# 3) 용도별 현금 유출 주기 계산
sorted_df['유출주기'] = sorted_df.groupby(['용도'])['출금일자'].diff().dt.days

# 4) 용도별 현금 유출액 및 주기 분석
fluctuation_analysis = sorted_df.groupby('용도').agg({
    '출금액': ['mean', 'std', 'count'],
    '유출주기': 'mean'
}).copy()

# 5) 표준편차 기준 위험도 분류 (표준편차가 클수록 위험도 증가하는 방식)
fluctuation_analysis.columns = ['평균출금액', '출금액표준편차(변동액)', '출금횟수', '평균유출주기(일)']

# 6) 평균유출주기 기준 위험도 평가
# 방법 : 평균유출주기가 긴 경우 위험도를 높게 설정 → 주기가 일정한데도 주기가 길면 위험하다고 판단하는 방식
bins = [0, fluctuation_analysis['평균유출주기(일)'].quantile(0.33), fluctuation_analysis['평균유출주기(일)'].quantile(0.66), float('inf')]
labels = ['높음', '보통', '낮음']
fluctuation_analysis['주기위험도'] = pd.cut(fluctuation_analysis['평균유출주기(일)'], bins=bins, labels=labels)

# 7) 출금횟수 2번 이상만 필터링
fluctuation_analysis_filtered = fluctuation_analysis[fluctuation_analysis['출금횟수'] >= 2].copy()

# 8) '평균출금액', '출금액표준편차', '평균유출주기' 소수점 2자리까지 출력
fluctuation_analysis_filtered['평균출금액'] = fluctuation_analysis_filtered['평균출금액'].round(2)
fluctuation_analysis_filtered['출금액표준편차(변동액)'] = fluctuation_analysis_filtered['출금액표준편차(변동액)'].round(2)
fluctuation_analysis_filtered['평균유출주기(일)'] = fluctuation_analysis_filtered['평균유출주기(일)'].round(2)

# 결과 출력
fluctuation_analysis_filtered

용도별 현금 유출 주기 및 변동액 계산 중


,평균출금액,출금액표준편차(변동액),출금횟수,평균유출주기(일),주기위험도
용도,,,,,
가수금반환,2259375.00,1482250.00,4,42.67,보통
가수금반환 외,3242500.00,306267.86,7,35.67,보통
간이영수증지급,3366017.29,2651319.20,96,10.23,높음
건강보험료,1061030.00,247215.50,24,46.26,보통
건강보험료 외,1120998.46,119402.63,13,30.42,보통
건물관리비,275320.00,22288.01,2,30.00,높음
건물관리비 외,280216.67,24605.91,9,44.88,보통
고용보험료,42700.45,23047.47,22,50.67,낮음
고용보험료 외,51590.77,39819.74,13,30.42,보통


In [20]:
# Model 평가 : 일치율(실제값과 예측치 간 동일한 비율)

# 1) Training Data와 Test Data 나누기 : 80% vs 20%
# 결측값이 있는 경우 위험도를 '높음'으로 설정
X = fluctuation_analysis_filtered[['평균출금액', '출금액표준편차(변동액)', '평균유출주기(일)']].copy()
y = fluctuation_analysis_filtered['주기위험도'].copy()

# 평균유출주기(일)에 결측값이 있는 경우 위험도를 '높음'으로 처리
X['평균유출주기(일)'] = X['평균유출주기(일)'].fillna(0)  # 결측값을 0으로 대체
y = y.where(X['평균유출주기(일)'] != 0, '높음')  # 평균 유출 주기가 0인 경우 위험도를 '높음'으로 설정

# 2) Training과 Test 데이터 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# 3) 예측 모델 (훈련 데이터의 평균 유출 주기를 기준으로 예측)
# 주기가 짧을수록 위험도가 '높음', 길수록 위험도가 '낮음'이므로 조건을 반대로 설정
y_pred = X_test['평균유출주기(일)'].apply(lambda x: '높음' if x <= X_train['평균유출주기(일)'].mean() else '낮음')

# 4) 일치율 평가 (accuracy)
accuracy = accuracy_score(y_test, y_pred)
accuracy_percentage = round(accuracy * 100, 2)  # 정확도를 소수점 2자리 %로 변환
print(f'주기위험도에 대한 일치율: {accuracy_percentage}%')

주기위험도에 대한 일치율: 88.89%


In [21]:
# '용도'를 Columns에 추가하고 기존 인덱스 제거
fluctuation_analysis_filtered = fluctuation_analysis_filtered.reset_index()

# '일치율' 열 추가
fluctuation_analysis_filtered['일치율'] = ''

# 마지막 행에 '일치율 결과'를 추가하고, 계산된 정확도(accuracy) 값을 넣음
fluctuation_analysis_filtered.loc['결과'] = ['', '', '', '', '', '', accuracy_percentage]

# 결과물 최종 경로
output_file = os.path.join(company_result_root, f'{company_name}_출금내역_유출_분석.xlsx')

# 엑셀 파일로 저장
fluctuation_analysis_filtered.to_excel(output_file, index=False)
print('1차 결과 : 유출 주기/변동액 분석 결과가 Excel 파일로 저장되었습니다.')

# 다시 엑셀 파일 불러오기
wb = load_workbook(output_file)
ws = wb.active

# 색상 지정
low_fill = PatternFill(start_color='32CD32', end_color='32CD32', fill_type='solid') # Limegreen
medium_fill = PatternFill(start_color='FFD700', end_color='FFD700', fill_type='solid') # Gold
high_fill = PatternFill(start_color='FF6347', end_color='FF6347', fill_type='solid') # Tomato

# '주기위험도' 열의 색상 적용
for row in range(2, ws.max_row):  # 2번째 행부터 반복(1번째 행 제외)
    risk_level = ws[f'F{row}'].value  # '주기위험도'가 있는 열 = Excel 상에서 F열
    if risk_level == '낮음':
        ws[f'F{row}'].fill = low_fill
    elif risk_level == '보통':
        ws[f'F{row}'].fill = medium_fill
    elif risk_level == '높음':
        ws[f'F{row}'].fill = high_fill

# 수정된 엑셀 파일 저장
wb.save(output_file)

print('2차 결과 : 주기위험도에 색상이 적용되었습니다.')

1차 결과 : 유출 주기/변동액 분석 결과가 Excel 파일로 저장되었습니다.
2차 결과 : 주기위험도에 색상이 적용되었습니다.
